# Pre-Processing Dataset

In [1]:
import os
import pandas as pd
import numpy as np

from scipy.stats import zscore

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelBinarizer

In [2]:
df = pd.read_csv("../0_Datasets/Farm-Flow/Farm-Flow.csv")

/tmp/ipykernel_48257/3848404070.py:1: DtypeWarning: Columns (6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../0_Datasets/Farm-Flow/Farm-Flow.csv")


In [3]:
df = df[[
#    'id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p', 'proto', 'service',
#    'duration', 'orig_bytes', 'resp_bytes', 'conn_state', 'local_orig',
#    'local_resp', 'missed_bytes', 'history', 'orig_pkts', 'orig_ip_bytes',
#    'resp_pkts', 'resp_ip_bytes', 'tunnel_parents', 'flow_duration',
    'fwd_pkts_tot', 'bwd_pkts_tot', 'fwd_data_pkts_tot', 'bwd_data_pkts_tot',
    'fwd_pkts_per_sec', 'bwd_pkts_per_sec', 'flow_pkts_per_sec', 'down_up_ratio',
    'fwd_header_size_tot', 'fwd_header_size_min', 'fwd_header_size_max',
    'bwd_header_size_tot', 'bwd_header_size_min', 'bwd_header_size_max',
#    'flow_FIN_flag_count', 'flow_SYN_flag_count', 'flow_RST_flag_count',
#    'fwd_PSH_flag_count', 'bwd_PSH_flag_count', 'flow_ACK_flag_count',
#    'fwd_URG_flag_count', 'bwd_URG_flag_count', 'flow_CWR_flag_count',
#    'flow_ECE_flag_count', 
    'fwd_pkts_payload.min', 'fwd_pkts_payload.max',
    'fwd_pkts_payload.tot', 'fwd_pkts_payload.avg', 'fwd_pkts_payload.std',
    'bwd_pkts_payload.min', 'bwd_pkts_payload.max', 'bwd_pkts_payload.tot',
    'bwd_pkts_payload.avg', 'bwd_pkts_payload.std', 'flow_pkts_payload.min',
    'flow_pkts_payload.max', 'flow_pkts_payload.tot', 'flow_pkts_payload.avg','flow_pkts_payload.std', 
#    'fwd_iat.min', 'fwd_iat.max', 'fwd_iat.tot','fwd_iat.avg', 'fwd_iat.std', 
#    'bwd_iat.min', 'bwd_iat.max', 'bwd_iat.tot','bwd_iat.avg', 'bwd_iat.std', 
#    'flow_iat.min', 'flow_iat.max', 'flow_iat.tot','flow_iat.avg', 'flow_iat.std', 'payload_bytes_per_second',
    'fwd_subflow_pkts', 'bwd_subflow_pkts', 'fwd_subflow_bytes',
    'bwd_subflow_bytes', 
# 'fwd_bulk_bytes', 'bwd_bulk_bytes', 'fwd_bulk_packets','bwd_bulk_packets', 'fwd_bulk_rate', 'bwd_bulk_rate', 
#    'active.min','active.max', 'active.tot', 'active.avg', 'active.std', 
#    'idle.min','idle.max', 'idle.tot', 'idle.avg', 'idle.std', 
#    'fwd_init_window_size','bwd_init_window_size', 'fwd_last_window_size', 'bwd_last_window_size',
    'traffic', 'is_attack']].copy()

In [4]:
features = [
    'fwd_pkts_tot', 'bwd_pkts_tot', 'fwd_data_pkts_tot', 'bwd_data_pkts_tot',
    'fwd_pkts_per_sec', 'bwd_pkts_per_sec', 'flow_pkts_per_sec', 'down_up_ratio',
    'fwd_header_size_tot', 'fwd_header_size_min', 'fwd_header_size_max',
    'bwd_header_size_tot', 'bwd_header_size_min', 'bwd_header_size_max',
    'fwd_pkts_payload.min', 'fwd_pkts_payload.max',
    'fwd_pkts_payload.tot', 'fwd_pkts_payload.avg', 'fwd_pkts_payload.std',
    'bwd_pkts_payload.min', 'bwd_pkts_payload.max', 'bwd_pkts_payload.tot',
    'bwd_pkts_payload.avg', 'bwd_pkts_payload.std', 'flow_pkts_payload.min',
    'flow_pkts_payload.max', 'flow_pkts_payload.tot', 'flow_pkts_payload.avg','flow_pkts_payload.std', 
    'fwd_subflow_pkts', 'bwd_subflow_pkts', 'fwd_subflow_bytes',
    'bwd_subflow_bytes', 'traffic', 'is_attack']

In [5]:
features_wo_target = [
    'fwd_pkts_tot', 'bwd_pkts_tot', 'fwd_data_pkts_tot', 'bwd_data_pkts_tot',
    'fwd_pkts_per_sec', 'bwd_pkts_per_sec', 'flow_pkts_per_sec', 'down_up_ratio',
    'fwd_header_size_tot', 'fwd_header_size_min', 'fwd_header_size_max',
    'bwd_header_size_tot', 'bwd_header_size_min', 'bwd_header_size_max',
    'fwd_pkts_payload.min', 'fwd_pkts_payload.max',
    'fwd_pkts_payload.tot', 'fwd_pkts_payload.avg', 'fwd_pkts_payload.std',
    'bwd_pkts_payload.min', 'bwd_pkts_payload.max', 'bwd_pkts_payload.tot',
    'bwd_pkts_payload.avg', 'bwd_pkts_payload.std', 'flow_pkts_payload.min',
    'flow_pkts_payload.max', 'flow_pkts_payload.tot', 'flow_pkts_payload.avg','flow_pkts_payload.std', 
    'fwd_subflow_pkts', 'bwd_subflow_pkts', 'fwd_subflow_bytes',
    'bwd_subflow_bytes']

In [6]:
df.shape

(1309887, 35)

In [7]:
# Remove duplicates
df = df.loc[~df.duplicated(subset=features)].reset_index(drop=True).copy()

In [8]:
df.shape

(170128, 35)

In [9]:
def remove_outliers(df,columns,n_std):
    for col in columns:
        print(f'[REMOVED OUTLIERS] {col}')
        
        mean = df[col].mean()
        sd = df[col].std()
        
        df = df[(df[col] <= mean+(n_std*sd))]
        
    return df

In [10]:
outliers = features_wo_target

df = remove_outliers(df, outliers, 3)

[REMOVED OUTLIERS] fwd_pkts_tot
[REMOVED OUTLIERS] bwd_pkts_tot
[REMOVED OUTLIERS] fwd_data_pkts_tot
[REMOVED OUTLIERS] bwd_data_pkts_tot
[REMOVED OUTLIERS] fwd_pkts_per_sec
[REMOVED OUTLIERS] bwd_pkts_per_sec
[REMOVED OUTLIERS] flow_pkts_per_sec
[REMOVED OUTLIERS] down_up_ratio
[REMOVED OUTLIERS] fwd_header_size_tot
[REMOVED OUTLIERS] fwd_header_size_min
[REMOVED OUTLIERS] fwd_header_size_max
[REMOVED OUTLIERS] bwd_header_size_tot
[REMOVED OUTLIERS] bwd_header_size_min
[REMOVED OUTLIERS] bwd_header_size_max
[REMOVED OUTLIERS] fwd_pkts_payload.min
[REMOVED OUTLIERS] fwd_pkts_payload.max
[REMOVED OUTLIERS] fwd_pkts_payload.tot
[REMOVED OUTLIERS] fwd_pkts_payload.avg
[REMOVED OUTLIERS] fwd_pkts_payload.std
[REMOVED OUTLIERS] bwd_pkts_payload.min
[REMOVED OUTLIERS] bwd_pkts_payload.max
[REMOVED OUTLIERS] bwd_pkts_payload.tot
[REMOVED OUTLIERS] bwd_pkts_payload.avg
[REMOVED OUTLIERS] bwd_pkts_payload.std
[REMOVED OUTLIERS] flow_pkts_payload.min
[REMOVED OUTLIERS] flow_pkts_payload.max
[REM

In [11]:
def zscore_normalization(df, cols):
    # Standardize the selected columns
    for col in cols:
        if col not in df.columns:
            print(f"[WARNING] {col} not found in DataFrame.")
            continue
        print(f"[Z-SCORE] {col}")
        df[col] = zscore(df[col])
    
    print("[DONE] Z-score Normalization")
    print("[INFO] Current Fields in the DataFrame:")
    return df

In [12]:
cols_to_zscore = features_wo_target

df = zscore_normalization(df, cols_to_zscore)

[Z-SCORE] fwd_pkts_tot
[Z-SCORE] bwd_pkts_tot
[Z-SCORE] fwd_data_pkts_tot
[Z-SCORE] bwd_data_pkts_tot
[Z-SCORE] fwd_pkts_per_sec
[Z-SCORE] bwd_pkts_per_sec
[Z-SCORE] flow_pkts_per_sec
[Z-SCORE] down_up_ratio
[Z-SCORE] fwd_header_size_tot
[Z-SCORE] fwd_header_size_min
[Z-SCORE] fwd_header_size_max
[Z-SCORE] bwd_header_size_tot
[Z-SCORE] bwd_header_size_min
[Z-SCORE] bwd_header_size_max
[Z-SCORE] fwd_pkts_payload.min
[Z-SCORE] fwd_pkts_payload.max
[Z-SCORE] fwd_pkts_payload.tot
[Z-SCORE] fwd_pkts_payload.avg
[Z-SCORE] fwd_pkts_payload.std
[Z-SCORE] bwd_pkts_payload.min
[Z-SCORE] bwd_pkts_payload.max
[Z-SCORE] bwd_pkts_payload.tot
[Z-SCORE] bwd_pkts_payload.avg
[Z-SCORE] bwd_pkts_payload.std
[Z-SCORE] flow_pkts_payload.min
[Z-SCORE] flow_pkts_payload.max
[Z-SCORE] flow_pkts_payload.tot
[Z-SCORE] flow_pkts_payload.avg
[Z-SCORE] flow_pkts_payload.std
[Z-SCORE] fwd_subflow_pkts
[Z-SCORE] bwd_subflow_pkts
[Z-SCORE] fwd_subflow_bytes
[Z-SCORE] bwd_subflow_bytes
[DONE] Z-score Normalization
[IN

In [13]:
df.shape

(107479, 35)

In [14]:
df.groupby('traffic')['traffic'].count()

traffic
Arp_Spoofing      2141
BotNet_DDOS         35
HTTP_Flood       34147
ICMP_Flood        2589
MQTT_Flood       15406
Normal           19209
Port_Scanning     2794
TCP_Flood         6361
UDP_Flood        24797
Name: traffic, dtype: int64

In [15]:
df.groupby('is_attack')['is_attack'].count()

is_attack
0    19209
1    88270
Name: is_attack, dtype: int64

In [16]:
undersample_count = 10000
category_to_undersample = 'HTTP_Flood'

category_count = df.groupby('traffic')['traffic'].count().loc[category_to_undersample]

undersampled_df = df[df['traffic'] == category_to_undersample].sample(n=undersample_count, random_state=42)
df_undersampled = pd.concat([df[df['traffic'] != category_to_undersample], undersampled_df])
df_undersampled = df_undersampled.sample(frac=1, random_state=42)

df = df_undersampled.copy()

In [17]:
undersample_count = 10000
category_to_undersample = 'MQTT_Flood'

category_count = df.groupby('traffic')['traffic'].count().loc[category_to_undersample]

undersampled_df = df[df['traffic'] == category_to_undersample].sample(n=undersample_count, random_state=42)
df_undersampled = pd.concat([df[df['traffic'] != category_to_undersample], undersampled_df])
df_undersampled = df_undersampled.sample(frac=1, random_state=42)

df = df_undersampled.copy()

In [18]:
undersample_count = 10000
category_to_undersample = 'UDP_Flood'

category_count = df.groupby('traffic')['traffic'].count().loc[category_to_undersample]

undersampled_df = df[df['traffic'] == category_to_undersample].sample(n=undersample_count, random_state=42)
df_undersampled = pd.concat([df[df['traffic'] != category_to_undersample], undersampled_df])
df_undersampled = df_undersampled.sample(frac=1, random_state=42)

df = df_undersampled.copy()

In [19]:
df = df.drop(['traffic'], axis=1)

In [20]:
df.groupby('is_attack')['is_attack'].count()

is_attack
0    19209
1    43920
Name: is_attack, dtype: int64

In [21]:
# Transform the categorical labels into binary format.
lb = LabelBinarizer()

# Transform the 'label' column to binary format.
df['is_attack'] = lb.fit_transform(df['is_attack'])

In [22]:
# Separate the features (X) and the target variable (y) from the DataFrame.
X = df.drop(['is_attack'], axis=1)  # Features
y = df['is_attack']  # Target variable

In [23]:
# Initialize Stratified K-Folds cross-validator with the specified number of splits.
skf = StratifiedKFold(n_splits=10, shuffle=False)

skf

StratifiedKFold(n_splits=10, random_state=None, shuffle=False)

In [24]:
# Iterate over each fold of the Stratified K-Folds cross-validator.
for train_index, test_index in skf.split(X, y):
    # Extract training and testing sets based on the generated indices.
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Reshape y_train and y_test to have a single column.
    reshaped_y_train = np.asarray(y_train).reshape(-1, 1)
    reshaped_y_test = np.asarray(y_test).reshape(-1, 1)

# Print the lengths of training and testing sets to verify the splits.
print('X_train length:', len(X_train))  # To check if splits worked
print('y_train length:', len(y_train))
print('X_test length:', len(X_test))
print('y_test length:', len(y_test))

X_train length: 56817
y_train length: 56817
X_test length: 6312
y_test length: 6312


# Saving Train and Test Datasets

In [25]:
# Save training data
train_data = pd.concat([X_train, y_train], axis=1)
train_data.to_csv('../0_Datasets/Farm-Flow/train.csv', index=False)

# Save testing data
test_data = pd.concat([X_test, y_test], axis=1)
test_data.to_csv('../0_Datasets/Farm-Flow/test.csv', index=False)